In [1]:
import unsloth
import transformers
import trl


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-08-31 16:04:14.300863: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-31 16:04:14.385150: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756636454.407316    5840 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756636454.414725    5840 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756636454.428002    5840 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:


import unsloth
from unsloth import FastModel
import torch
import os

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3n-E4B-it-unsloth-bnb-4bit",
    "unsloth/gemma-3n-E2B-it-unsloth-bnb-4bit",
    "unsloth/gemma-3n-E4B-unsloth-bnb-4bit",
    "unsloth/gemma-3n-E2B-unsloth-bnb-4bit",
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",
] 

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3n-E4B-it",
    dtype = None, 
    max_seq_length = 1024, 
    load_in_4bit = True,  
    full_finetuning = False
)


==((====))==  Unsloth 2025.8.10: Fast Gemma3N patching. Transformers: 4.56.0.
   \\   /|    NVIDIA GeForce RTX 3050 Laptop GPU. Num GPUs = 1. Max memory: 3.68 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3N does not support SDPA - switching to fast eager.


model-00001-of-00003.safetensors:   0%|          | 0.00/3.72G [00:00<?, ?B/s]

In [ ]:
# !pip install unsloth

# !pip install --no-deps --upgrade transformers
# !pip install --no-deps --upgrade timm

In [ ]:
# pip install unsloth_zoo

In [ ]:
from transformers import TextStreamer
import gc

def do_gemma_3n_inference(model, tokenizer, messages, max_new_tokens = 128):
    
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt = True, 
        tokenize = True,
        return_dict = True,
        return_tensors = "pt",
    ).to("cuda")
    
    output_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=1.0,
        top_p=0.95,
        top_k=64,
        do_sample=True
    )

    generated_text = tokenizer.decode(
        output_ids[0][inputs['input_ids'].shape[-1]:], 
        skip_special_tokens=True
    )
    
    # Cleanup to reduce VRAM usage
    del inputs
    torch.cuda.empty_cache()
    gc.collect()

    return generated_text

In [ ]:
import torch
import gc

import torch._dynamo
torch._dynamo.config.suppress_errors = True
torch._dynamo.config.cache_size_limit = 512

In [ ]:
def create_unified_training_data(df1, df2):
    """Combine both datasets into a unified training format"""
    
    print("🔄 Creating unified training dataset...")
    
    unified_data = []
    
    # Process Dataset 1 (Detailed)
    print("Processing detailed dataset...")
    for _, row in df1.iterrows():
        # Basic recommendation
        basic_prompt = (
            f"N: {row['Nitrogen']}, P: {row['Phosphorus']}, K: {row['Potassium']}, "
            f"temperature: {row['Temperature']:.2f}, humidity: {row['Humidity']:.2f}, "
            f"pH: {row['pH_Value']:.2f}, rainfall: {row['Rainfall']:.2f}"
        )
        basic_response = f"Recommended crop: {row['Crop']}"
        
        # Detailed recommendation with soil type and variety
        detailed_prompt = (
            f"N: {row['Nitrogen']}, P: {row['Phosphorus']}, K: {row['Potassium']}, "
            f"temperature: {row['Temperature']:.2f}, humidity: {row['Humidity']:.2f}, "
            f"pH: {row['pH_Value']:.2f}, rainfall: {row['Rainfall']:.2f}, "
            f"soil_type: {row['Soil_Type']}"
        )
        detailed_response = (
            f"Recommended crop: {row['Crop']}. "
            f"Suitable soil type: {row['Soil_Type']}. "
            f"Recommended variety: {row['Variety']}. "
            f"This combination is optimal for the given soil and climate conditions."
        )
        
        # Add both variations
        unified_data.append({"prompt": basic_prompt, "response": basic_response, "source": "dataset1_basic"})
        unified_data.append({"prompt": detailed_prompt, "response": detailed_response, "source": "dataset1_detailed"})
    
    # Process Dataset 2 (Simple)
    print("Processing simple dataset...")
    for _, row in df2.iterrows():
        prompt = (
            f"N: {row['N']}, P: {row['P']}, K: {row['K']}, "
            f"temperature: {row['temperature']:.2f}, humidity: {row['humidity']:.2f}, "
            f"pH: {row['ph']:.2f}, rainfall: {row['rainfall']:.2f}"
        )
        response = f"Recommended crop: {row['label']}"
        
        unified_data.append({"prompt": prompt, "response": response, "source": "dataset2"})
    
    print(f"✅ Created {len(unified_data)} training examples")
    return unified_data

In [ ]:

df2 = pd.read_csv('Crop_recommendation.csv')
df1  = pd.read_csv('sensor_Crop_Dataset.csv')
unified_data = create_unified_training_data(df1, df2)